# This notebook is part of the AI Digital Twin project.
# It is designed to be run in a Jupyter Notebook environment.
# The code is structured to process ICU data, extract features, and train models.
# download the MIMIC-III data files before proceeding

Requirements

In [ ]:
#Install required packages:
%pip install -r requirements.txt

*** Setup and paths ***
Download MIMIC-III csv data files. 
ICUSTAYS, PRESCRIPTIONS, MICROBIOLOGYEVENTS, CHARTEVENTS, LABEVENTS files are used. save these files to input folder and provide the path in below setup code block.

In [1]:
# Core
import os, json, math, gc, random, warnings
warnings.filterwarnings("ignore")

# Data / ML
import pandas as pd
import numpy as np

# Paths
DATA_DIR = "."

RAW = {
    "ICUSTAYS": f"input/ICUSTAYS.csv.gz",
    "CHARTEVENTS": f"input/CHARTEVENTS.csv.gz",   # or .csv
    "LABEVENTS": f"input/LABEVENTS.csv.gz",
}

OUT = f"{DATA_DIR}/output"
os.makedirs(OUT, exist_ok=True)

LABELS = {
    "SEPSIS": f"{OUT}/sepsis_labels.csv",
    "CONTROL": f"{OUT}/control_labels.csv",
}

SEED = 42
np.random.seed(SEED); random.seed(SEED)

# Quick sanity
for k,v in {**RAW, **LABELS}.items():
    print(f"{k}: {'OK' if os.path.exists(v) else 'MISSING'} → {v}")


ICUSTAYS: OK → input/ICUSTAYS.csv.gz
CHARTEVENTS: OK → input/CHARTEVENTS.csv.gz
LABEVENTS: OK → input/LABEVENTS.csv.gz
SEPSIS: OK → ./output/sepsis_labels.csv
CONTROL: OK → ./output/control_labels.csv


Filter CHARTEVENTS to vital ITEMIDs 

In [ ]:
ITEM_IDS = {
    "Heart Rate": [211],
    "Systolic BP": [51],
    "Diastolic BP": [8368],
    "Mean BP": [456],
    "Respiratory Rate": [618],
    "SpO2": [646],
    "Temperature": [223761],
}

use_itemids = sorted({i for ids in ITEM_IDS.values() for i in ids})
usecols = ["ICUSTAY_ID","ITEMID","CHARTTIME","VALUENUM"]

# If you already have filtered_chartevents.csv.gz, skip this cell.
ce = pd.read_csv(RAW["CHARTEVENTS"], usecols=usecols, parse_dates=["CHARTTIME"])
ce = ce[ce["ITEMID"].isin(use_itemids)]
ce.to_csv(f"{OUT}/filtered_chartevents.csv.gz", index=False, compression="gzip")
print(ce.shape, "[INFO] saved to output/filtered_chartevents.csv.gz")
del ce; gc.collect()


(18244797, 4) → saved to output/filtered_chartevents.csv.gz


23

2) Vitals feature extraction (12h window) - sepsis & controls

In [ ]:
from tqdm import tqdm

def extract_vitals(labels_file, out_file, chart_file=f"{OUT}/filtered_chartevents.csv.gz"):
    chart = pd.read_csv(chart_file, parse_dates=["CHARTTIME"])
    labels = pd.read_csv(labels_file)
    labels["SEPSIS_ONSET"] = pd.to_datetime(labels["SEPSIS_ONSET"])

    feats = []
    for _, r in tqdm(labels.iterrows(), total=len(labels)):
        icu, onset = r["ICUSTAY_ID"], r["SEPSIS_ONSET"]
        start = onset - pd.Timedelta(hours=12)
        win = chart[(chart["ICUSTAY_ID"]==icu) &
                    (chart["CHARTTIME"]>=start) &
                    (chart["CHARTTIME"]<=onset)]

        row = {"ICUSTAY_ID": icu}
        for name, ids in ITEM_IDS.items():
            vals = win[win["ITEMID"].isin(ids)]["VALUENUM"].dropna()
            row[f"{name}_mean"]  = vals.mean() if not vals.empty else np.nan
            row[f"{name}_std"]   = vals.std()  if not vals.empty else np.nan
            row[f"{name}_delta"] = (vals.iloc[-1]-vals.iloc[0]) if len(vals)>1 else np.nan
        feats.append(row)

    df = pd.DataFrame(feats)
    df.to_csv(out_file, index=False)
    print(f"[INFO] Saved {out_file} | shape={df.shape}")

extract_vitals(LABELS["SEPSIS"],  f"{OUT}/features_robust.csv")
extract_vitals(LABELS["CONTROL"], f"{OUT}/control_features_robust.csv")


3) Lab feature extraction (12h window; join Labevents and icustays)

In [ ]:
LAB_ITEMS = {  # MIMIC-III codes
    "Lactate": [50813],
    "WBC": [51300],
    "Creatinine": [50912],
    "Bilirubin": [50885],
    "Platelets": [51265],
}

def extract_labs(labels_file, out_file):
    labs = pd.read_csv(RAW["LABEVENTS"], usecols=["HADM_ID","ITEMID","CHARTTIME","VALUENUM"],
                       parse_dates=["CHARTTIME"])
    icu = pd.read_csv(RAW["ICUSTAYS"], usecols=["ICUSTAY_ID","HADM_ID","INTIME","OUTTIME"],
                      parse_dates=["INTIME","OUTTIME"])
    labs = labs.merge(icu, on="HADM_ID", how="inner")
    # keep only labs within the ICU stay
    labs = labs[(labs["CHARTTIME"]>=labs["INTIME"]) & (labs["CHARTTIME"]<=labs["OUTTIME"])]
    labels = pd.read_csv(labels_file); labels["SEPSIS_ONSET"] = pd.to_datetime(labels["SEPSIS_ONSET"])

    rows=[]
    for _, r in tqdm(labels.iterrows(), total=len(labels)):
        icustay, onset = r["ICUSTAY_ID"], r["SEPSIS_ONSET"]
        start = onset - pd.Timedelta(hours=12)
        win = labs[(labs["ICUSTAY_ID"]==icustay) &
                   (labs["CHARTTIME"]>=start) &
                   (labs["CHARTTIME"]<=onset)]
        row={"ICUSTAY_ID": icustay}
        for name, ids in LAB_ITEMS.items():
            v = win[win["ITEMID"].isin(ids)]["VALUENUM"].dropna()
            row[f"{name}_mean"]=v.mean() if not v.empty else np.nan
            row[f"{name}_std"] =v.std()  if not v.empty else np.nan
            row[f"{name}_delta"]=(v.iloc[-1]-v.iloc[0]) if len(v)>1 else np.nan
        rows.append(row)

    df=pd.DataFrame(rows)
    df.to_csv(out_file, index=False)
    print(f"[INFO] Saved {out_file} | shape={df.shape}")

extract_labs(LABELS["SEPSIS"],  f"{OUT}/lab_features_sepsis.csv")
extract_labs(LABELS["CONTROL"], f"{OUT}/lab_features_controls.csv")


4) Merge vitals + labs + labels -> model ready csv

In [ ]:
sev = pd.read_csv(f"{OUT}/features_robust.csv").merge(
    pd.read_csv(f"{OUT}/lab_features_sepsis.csv"), on="ICUSTAY_ID", how="left"
).assign(label=1)

ctl = pd.read_csv(f"{OUT}/control_features_robust.csv").merge(
    pd.read_csv(f"{OUT}/lab_features_controls.csv"), on="ICUSTAY_ID", how="left"
).assign(label=0)

data = pd.concat([sev, ctl], ignore_index=True)
print("[INFO] Before impute:", data.shape, "missing:", data.isna().sum().sum())
data = data.fillna(data.mean(numeric_only=True))
data = data.sample(frac=1, random_state=SEED).reset_index(drop=True)
data.to_csv(f"{OUT}/merged_dataset_with_labs.csv", index=False)
data.head(3)


5) Train & evaluate XGBoost (save model)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report
from xgboost import XGBClassifier
import joblib, os

df = pd.read_csv(f"{OUT}/merged_dataset_with_labs.csv")
X = df.drop(columns=["ICUSTAY_ID","label"], errors="ignore")
y = df["label"]

X_tr, X_te, y_tr, y_te = train_test_split(X, y, stratify=y, test_size=0.2, random_state=SEED)

clf = XGBClassifier(n_estimators=200, max_depth=4, learning_rate=0.05, subsample=0.9,
                    colsample_bytree=0.9, random_state=SEED, n_jobs=-1)
clf.fit(X_tr, y_tr)
p = clf.predict_proba(X_te)[:,1]

print("[INFO] AUROC :", round(roc_auc_score(y_te, p), 4))
print("[INFO] AUPRC :", round(average_precision_score(y_te, p), 4))
print(classification_report(y_te, p>0.5))

os.makedirs("models", exist_ok=True)
joblib.dump(clf, "models/xgb_with_labs.joblib"); print("Saved models/xgb_with_labs.joblib")


6) SHAP (summary plot +3 force plats as html)

In [ ]:
import shap, matplotlib.pyplot as plt
explainer = shap.Explainer(clf)
sv = explainer(X)  # on full feature set for global importance

shap.summary_plot(sv, X, show=False)
plt.tight_layout(); plt.savefig(f"{OUT}/shap_summary_with_labs.png", dpi=200, bbox_inches="tight"); plt.close()

shap.initjs()
for i in range(3):
    fp = shap.force_plot(explainer.expected_value, sv[i].values, X.iloc[i], feature_names=X.columns)
    with open(f"{OUT}/force_plot_with_labs_{i}.html", "w") as f: f.write(fp.html())
print("[INFO] Saved SHAP outputs.")


7) Build time-series tensors (vitals-only) for GRU-D

In [ ]:
# Rebuild from filtered_chartevents for both labels
chart = pd.read_csv(f"{OUT}/filtered_chartevents.csv.gz", parse_dates=["CHARTTIME"])
lab_all = []
lab_all.append(pd.read_csv(LABELS["SEPSIS"]).assign(label=1))
lab_all.append(pd.read_csv(LABELS["CONTROL"]).assign(label=0))
labels_all = pd.concat(lab_all, ignore_index=True)
labels_all["SEPSIS_ONSET"] = pd.to_datetime(labels_all["SEPSIS_ONSET"])

FEATURES = list(ITEM_IDS.keys())
WINDOW_HOURS = 12
D = len(FEATURES)

X_list, M_list, Y_list = [], [], []

for _, r in tqdm(labels_all.iterrows(), total=len(labels_all)):
    icu, onset = r["ICUSTAY_ID"], r["SEPSIS_ONSET"]
    start = onset - pd.Timedelta(hours=WINDOW_HOURS)
    df = chart[(chart["ICUSTAY_ID"]==icu) & (chart["CHARTTIME"]>=start) & (chart["CHARTTIME"]<=onset)].copy()
    df["hour"] = ((df["CHARTTIME"]-start).dt.total_seconds()//3600).astype(int)

    Xp = np.full((WINDOW_HOURS,D), np.nan, dtype=float)
    Mp = np.zeros_like(Xp)

    for j,f in enumerate(FEATURES):
        ids = ITEM_IDS[f]
        for h in range(WINDOW_HOURS):
            vals = df[(df["ITEMID"].isin(ids)) & (df["hour"]==h)]["VALUENUM"].dropna()
            if not vals.empty:
                Xp[h,j] = vals.mean(); Mp[h,j] = 1

    X_list.append(Xp); M_list.append(Mp); Y_list.append(r["label"])

X_ts = np.array(X_list); M_ts = np.array(M_list); y_ts = np.array(Y_list)
np.save(f"{OUT}/timeseries_X.npy", X_ts)
np.save(f"{OUT}/timeseries_M.npy", M_ts)
np.save(f"{OUT}/timeseries_labels.npy", y_ts)
X_ts.shape, M_ts.shape, y_ts.shape


8) Train GRU-D (CPU friendly)

In [ ]:
import torch, torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score

# Minimal GRU-D style block (mean-impute + learned gates)
class GRUD(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.gamma_x = nn.Parameter(torch.ones(input_size))
        self.z = nn.Linear(input_size*3, hidden_size)
        self.r = nn.Linear(input_size*3, hidden_size)
        self.h_tilde = nn.Linear(input_size*3, hidden_size)
        self.h_out = nn.Linear(hidden_size, 1)
        self.input_means = None

    def forward(self, X, M):
        B,T,D = X.shape
        if self.input_means is None:
            raise ValueError("Set model.input_means tensor first")
        device = X.device
        h = torch.zeros(B, self.hidden_size, device=device)
        Xmean = self.input_means.unsqueeze(0).unsqueeze(0).expand(B,T,D)

        for t in range(T):
            x_t, m_t = X[:,t], M[:,t]
            x_hat = m_t*x_t + (1-m_t)*Xmean[:,t]
            x_gamma = torch.exp(-self.gamma_x*(1-m_t))
            x_tilde = x_gamma*x_hat + (1-x_gamma)*Xmean[:,t]
            inp = torch.cat([x_tilde, Xmean[:,t], m_t], dim=1)
            zt = torch.sigmoid(self.z(inp)); rt = torch.sigmoid(self.r(inp))
            h_t = torch.tanh(self.h_tilde(inp))
            h = (1-zt)*h + zt*h_t
        out = torch.sigmoid(self.h_out(h)).squeeze(1)
        return out

# Load tensors
X = np.load(f"{OUT}/timeseries_X.npy")
M = np.load(f"{OUT}/timeseries_M.npy")
y = np.load(f"{OUT}/timeseries_labels.npy")

# Torch tensors
X = torch.tensor(np.nan_to_num(X, nan=0.0), dtype=torch.float32)
M = torch.tensor(M, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

X_tr, X_va, M_tr, M_va, y_tr, y_va = train_test_split(X, M, y, test_size=0.2, stratify=y, random_state=SEED)

model = GRUD(input_size=X.shape[2], hidden_size=64)
train_mean = torch.tensor(np.nanmean(X_tr.numpy(), axis=(0,1)), dtype=torch.float32)
train_mean = torch.nan_to_num(train_mean, nan=0.0)
model.input_means = train_mean

opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

EPOCHS = 20; BATCH=64
for ep in range(1, EPOCHS+1):
    model.train(); idx = torch.randperm(X_tr.size(0)); epoch_loss=0.0
    for i in range(0, X_tr.size(0), BATCH):
        b = idx[i:i+BATCH]
        opt.zero_grad()
        out = model(X_tr[b], M_tr[b])
        loss = loss_fn(out, y_tr[b])
        loss.backward(); opt.step()
        epoch_loss += loss.item()
    # val
    model.eval()
    with torch.no_grad():
        p = model(X_va, M_va).numpy()
        auroc = roc_auc_score(y_va.numpy(), p)
        auprc = average_precision_score(y_va.numpy(), p)
    print(f"Epoch {ep:02d} | Loss {epoch_loss:.3f} | AUROC {auroc:.3f} | AUPRC {auprc:.3f}")

torch.save(model.state_dict(), "models/grud_model.pt"); print("Saved models/grud_model.pt")


9) GRU-D evaluation plots (ROC/PR) + risk trajetories 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve

model.eval()
with torch.no_grad():
    p = model(X_va, M_va).numpy(); yy = y_va.numpy()

auroc = roc_auc_score(yy, p)
auprc = average_precision_score(yy, p)
fpr,
